<a href="https://colab.research.google.com/github/drihn/CCRNFLRL_COM221ML/blob/main/exercise2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 2

In [584]:
from math import sqrt, log
import pandas as pd

In [585]:
# Make sure your spreadsheet is saved in your google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Step 1: Enter your data

Add a new row after each pull: Pull number, Machine played ("A", "B", "C"), and Reward earned.

In [586]:
# To load data from a Google Sheet, you need the SHEET_ID and SHEET_NAME.
# You can find the SHEET_ID in the URL of your Google Sheet, between "/d/" and "/edit".
# The SHEET_NAME is usually "Sheet1" by default, but you should verify the name of the sheet containing your data.
# Ensure your Google Sheet is shared with "Anyone with the link" as Viewer.

SHEET_ID = '11NxyRM128KhUaTYuYragO3dk1ctrAW_S6Fa-DpD1oHA'
SHEET_NAME = 'UCB'
url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'

try:
    data = pd.read_csv(url)
    print("Data loaded successfully:")
except Exception as e:
    print(f"Error loading data: {e}")
    print("Please ensure the sheet is publicly accessible or shared with 'Anyone with the link' as Viewer.")

Data loaded successfully:


# Step 2: Calculate UCB

In [587]:
def calculate_ucb(df):
    results = []
    total_pulls = df.dropna(subset=['Machine'])['Pull #'].max()
    print(total_pulls)
    if pd.isna(total_pulls):
        total_pulls = 0

    machines = sorted(df["Machine"].dropna().unique())

    for machine in machines:
        machine_data = df[df["Machine"] == machine].dropna(subset=['Pull #', 'Reward'])
        n_j = len(machine_data)
        avg_reward = machine_data["Reward"].mean() if n_j > 0 else 0
        exploration_bonus = sqrt((2 * log(total_pulls)) / n_j) if n_j > 0 and total_pulls > 0 else float("inf")
        ucb_score = avg_reward + exploration_bonus
        results.append({
            "Machine": machine,
            "UCB Score": round(ucb_score, 4)
        })
    return pd.DataFrame(results).sort_values(by="UCB Score", ascending=False)


# Step 3: Calculate and display


In [588]:
ucb_table = calculate_ucb(data)
print("📊 UCB Results Table")
print(ucb_table)

100
📊 UCB Results Table
  Machine  UCB Score
0       A     2.5962
1       B     1.2390
2       C     1.2390


In [589]:
last_input_index = data.dropna(subset=['Machine']).index.max()

if pd.isna(last_input_index):
    print("No machine data has been inputted yet to update UCB scores.")
else:

    ucb_a_score = ucb_table[ucb_table['Machine'] == 'A']['UCB Score'].iloc[0] if 'A' in ucb_table['Machine'].values else None
    ucb_b_score = ucb_table[ucb_table['Machine'] == 'B']['UCB Score'].iloc[0] if 'B' in ucb_table['Machine'].values else None
    ucb_c_score = ucb_table[ucb_table['Machine'] == 'C']['UCB Score'].iloc[0] if 'C' in ucb_table['Machine'].values else None

    # Update the corresponding UCB score columns in the last inputted row
    if 'UCB Score A' in data.columns and ucb_a_score is not None:
        data.loc[last_input_index, 'UCB Score A'] = ucb_a_score
    else:
         print("Warning: Could not update 'UCB Score A'. Column not found or Machine A score not available.")

    if 'UCB Score B' in data.columns and ucb_b_score is not None:
        data.loc[last_input_index, 'UCB Score B'] = ucb_b_score
    else:
        print("Warning: Could not update 'UCB Score B'. Column not found or Machine B score not available.")

    if 'UCB Score C' in data.columns and ucb_c_score is not None:
        data.loc[last_input_index, 'UCB Score C'] = ucb_c_score
    else:
        print("Warning: Could not update 'UCB Score C'. Column not found or Machine C score not available.")

    print(f"UCB scores for Machines A, B, and C updated in the last inputted row (index: {last_input_index}) of the data DataFrame.")
    # Display the updated last row with the relevant columns
    display(data.loc[[last_input_index], ['Pull #', 'Machine', 'Reward', 'UCB Score A', 'UCB Score B', 'UCB Score C']])

UCB scores for Machines A, B, and C updated in the last inputted row (index: 99) of the data DataFrame.


,Pull #,Machine,Reward,UCB Score A,UCB Score B,UCB Score C
99,100,A,0.0,2.5962,1.239,1.239


In [590]:
best_machine = ucb_table.iloc[0]["Machine"]
print(f"\n🎯 Recommended next machine to play: {best_machine}")


🎯 Recommended next machine to play: A


Based on the data, Machine A is the only profitable one, showing a positive average pull value (2.17), while Machines B and C have no recorded profit (average = 0).